## **1**. Imports

In [ ]:
ON_COLAB=False
if ON_COLAB:
    from google.colab import drive
    drive.mount('/content/drive',force_remount=True)
    BASEDIR='/content/drive/My Drive/MentalHealthShared/'
    PYTHONDIR=BASEDIR+'src'
    RESULTSDIR=BASEDIR+'results/'
    MODELSDIR=BASEDIR+'model/'
    DATADIR=BASEDIR+'data/'
    import models
    import pytorchtools
    from utils import reset_seeds, count_parameters, evaluate, train_over_nepochs, createTensorDataset
    from training_functions import load_df, compute_bin_weights, save_stats_tensors, load_stats_tensors, get_znorm_params, get_subreddit_range, split_indices, get_subreddit_weights, get_baselines_df, WeightedL1Loss, WeightedMSELoss, grid_search_train
else:
    import os
    BASEDIR = os.getcwd() + "/"
    dirs = ["results","model","data"]
    for dirc in dirs:
        if dirc not in os.listdir(): 
            os.makedirs(os.path.join(BASEDIR,dirc))
    PYTHONDIR=BASEDIR+'src/'
    RESULTSDIR=BASEDIR+'results/'
    MODELSDIR=BASEDIR+'model/'
    DATADIR=BASEDIR+'data/'
    from src import models
    from src import pytorchtools
    from src.utils import reset_seeds, count_parameters, evaluate, train_over_nepochs, createTensorDataset
    from src.training_functions import load_df, compute_bin_weights, save_stats_tensors, load_stats_tensors, get_znorm_params, get_subreddit_range, split_indices, get_subreddit_weights, get_baselines_df, WeightedL1Loss, WeightedMSELoss, grid_search_train

In [ ]:
# DEFINE SUBREDDITS
SUBREDDITS  = ['Anxiety','bipolar','depression','SuicideWatch'] 

subreddit2title = {'depression':'DEP','suicidewatch':'SUI','anxiety':'ANX','bipolar':'BIP'}

EXTENSION = '.parquet'
STRATIFIED = True  # If true, will apply a stratified K-fold cross-validation to the dataset.
TRAIN = True # If true, will train the models

USE_GRU = True # If true, will use the GRU RNN model in training
USE_XGB = True # If true, will use the XGBoost model in training

GRID_SEARCH_XGB = False # If true, will do a grid search for the XGBoost model
PLOT_XGB = False # If true, will plot the results found by the XGBoost model(Only available if TRAIN = True)

TEST  = False # If true, will test the models

ZNORMALIZE = False # If true, will apply a z-normalization to the dataset


KEEP_TEXT = True # Only True if using Section 6 for Case Study


assert TRAIN ^ TEST # either TRAIN or TEST

FILTERED=True # If true will use filtered seq_len for the threads
BIN_WIDTH=0.2 # Controls the width of the bins used to calculate the weighted L1 loss
MIN_VALUE = -1 # Controls the minimum output value(set to -1)


INCLUDE_TARGET = 0


MAX_BRANCH_LEN = 16   # not including authors' last comment
MAX_THREAD_LEN = 64   # not including authors' last comment

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device == torch.device('cuda'):
    print(f"Device successfully set to cuda")
else:
    print("WARNING! DEVICE IS NOT SET TO CUDA")

print(torch.__version__)

In [ ]:
import sys
sys.path.append(PYTHONDIR)
import random
import pickle
import time
import pdb
import importlib
import itertools
import pprint
import copy
import os

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.metrics import mean_squared_error

from tqdm.notebook import tqdm
from scipy import stats

import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, TensorDataset, DataLoader, random_split, Sampler, SubsetRandomSampler, Subset

from xgboost import XGBRegressor, Booster, DMatrix


# **2**. Creating a RedditDataset or TensorDataset instance

Instead of using a custom Dataset class (former ```RedditDataset```), we store the data required by the experiments in a ```TensorDataset``` to speed up the retrieval of the batchs.

```set_threads``` is a list of indexes containing the threads to be used as a dataset.

## **2.1** Build the source data

* Build the source data in order to predict the EmT based on the sequence of comments in a thread

In [ ]:
suffix='_distilbert_filtered_posts' + EXTENSION
df_list = {subreddit:load_df(DATADIR+subreddit+suffix, MAX_THREAD_LEN) \
           for subreddit in SUBREDDITS}

post_df = pd.concat((df_list[subreddit] for subreddit in SUBREDDITS), keys=SUBREDDITS)
del df_list

In [ ]:
if not KEEP_TEXT:
  post_df = post_df[['created_utc', 'seq_len','score', 'features', 'filtered_seqlen','valid_branches']]

# new strategy to construct observations: follow branches of every discussion tree 
#post_df.drop(columns=['valid_branches'],inplace=True)

if FILTERED:
    # dropna on filtered_seqlen, then replace seq_len by filtered_seqlen
    post_df.dropna(subset=['filtered_seqlen'], inplace=True)
    post_df.filtered_seqlen = post_df.filtered_seqlen.astype(int)

    post_df.drop(columns='seq_len',inplace=True)
    post_df.rename(columns={'filtered_seqlen':'seq_len'},inplace=True)
else:
    post_df.drop(columns='filtered_seqlen',inplace=True)

print(f'Fraction of threads that had to be truncated: {(post_df.seq_len>(MAX_THREAD_LEN+1)).mean()}')


In [ ]:
if ZNORMALIZE:
  prefix = ''
else:
  prefix = 'unnorm_'

if TRAIN:
  if ZNORMALIZE:
    src_m, src_s = get_znorm_params(post_df)
  else:
    shape = [1,post_df.iloc[0].features.shape[-1]]
    src_m = torch.zeros(shape)
    src_s = torch.ones(shape)

  save_stats_tensors(src_m,src_s,f'{BASEDIR}data/{prefix}')
  
if TEST:
  src_m, src_s = load_stats_tensors(f'{BASEDIR}data/{prefix}')

score_m = float(src_m[0,-2])
score_s = float(src_s[0,-2])

print(f'Average score in dataset is {score_m}')

subreddit2range = get_subreddit_range(post_df)
print(subreddit2range[SUBREDDITS[0]])

# TODO: define constants upfront
suffix = "random"
if STRATIFIED:
  suffix += '_strat'

# TODO: place indexing in a new function and explain what it does
# get indices
if TRAIN:
  if len(SUBREDDITS) > 1:
      train_inds, valid_inds, test_inds  = split_indices(post_df, STRATIFIED, MIN_VALUE, BIN_WIDTH)
  else:
    subreddit = SUBREDDITS[0]
    with open(f'{DATADIR}{subreddit}_{suffix}_splits.pkl','rb') as infile:
      splits = pickle.load(infile)
      train_locs =  [(subreddit,loc) for loc in splits[0]]
      valid_locs =  [(subreddit,loc) for loc in splits[1]]
      test_locs  =  [(subreddit,loc) for loc in splits[2]]

    train_inds = post_df.index.get_indexer_for(train_locs)
    valid_inds = post_df.index.get_indexer_for(valid_locs)
    test_inds  = post_df.index.get_indexer_for(test_locs)


if TEST:
  train_locs = []
  valid_locs = []
  test_locs  = []
  for subreddit in SUBREDDITS:
      with open(f'{DATADIR}{subreddit}_{suffix}_splits.pkl','rb') as infile:
        splits = pickle.load(infile)
        test_locs += [(subreddit,loc) for loc in splits[2]]

  # extract instances
  print(len(post_df))
  post_df = post_df.loc[test_locs]
  print(len(post_df))

# compute weights for Weighted L1 Loss
subreddit2weights = get_subreddit_weights(post_df, BIN_WIDTH,MIN_VALUE,device)
print(subreddit2weights)


In [ ]:
%%time

# TODO: place the dataset generation in a function


print('Creating src')

print('Creating y')
y = torch.Tensor(post_df.apply(lambda p: p.score[p.seq_len-1], axis=1).values)

print('Creating src_len_series')
src_len_series = post_df.seq_len-1
max_length=MAX_THREAD_LEN

if USE_GRU:
    src = nn.utils.rnn.pad_sequence(
    [ p.features[:min(MAX_THREAD_LEN,p.seq_len-1),:] for index, p in post_df.iterrows()], batch_first=True)
    # src = nn.utils.rnn.pad_sequence(
    # [ (p.features[:min(MAX_THREAD_LEN,p.seq_len-1),:]-src_m)/src_s for index, p in post_df.iterrows()], batch_first=True)
    print(f"GRU src tensor size: {src.size()}")

if USE_XGB:      
    src_xgb = torch.cat((
        torch.cat([torch.mean((p.features[:min(MAX_THREAD_LEN,p.seq_len-1),:]-src_m)/src_s,keepdims=True,dim=0) for index, p in post_df.iterrows()],axis=0),
        torch.cat([torch.max((p.features[:min(MAX_THREAD_LEN,p.seq_len-1),:]-src_m)/src_s,keepdims=True,dim=0)[0] for index, p in post_df.iterrows()],axis=0)
        ),1)
    print(f"XGB src tensor size: {src_xgb.size()}")


print(f'y tensor size: {y.size()}')


if INCLUDE_TARGET:
  tgt = nn.utils.rnn.pad_sequence(
    [(p.features[b[-2]]-src_m)/src_s for index, p in post_df.iterrows() for b in p.valid_branches],
  batch_first=True)
else:
  tgt = None

  # clean up memory
  #if (not TEST) and (not KEEP_TEXT):
  if (not KEEP_TEXT):
    del post_df

  print('Creating dataset')
  if USE_GRU:
    #dataset = createTensorDataset(src, src_len_series, y, tgt=tgt, max_length=max_length) # all threads, lim 63 comments
    dataset = createTensorDataset(src, src_len_series, y, max_length=max_length) # all threads, lim 63 comments
    del src
  if USE_XGB:
    dataset_xgb = createTensorDataset(src_xgb, src_len_series, y, max_length=max_length)
    del src_xgb
  
  del src_len_series, tgt, y

In [ ]:
if USE_GRU:
  batch = dataset[0]
  EMBEDDING_DIM = batch[0].shape[-1]
  print(f"embedding dimension for GRU: {EMBEDDING_DIM}")
if USE_XGB:
  batch = dataset_xgb[0]
  EMBEDDING_DIM_XGB = batch[0].shape[-1]
  print(f"embedding dimension for XGB: {EMBEDDING_DIM_XGB}")

## **2.2** Create training, validation and test sets

In [ ]:
#%%time

print("Creating dataloaders...")
if TRAIN:
  if USE_XGB:
    train_loader_xgb = DataLoader(Subset(dataset_xgb,train_inds), batch_size=len(train_inds), shuffle=True, num_workers=0)
    valid_loader_xgb = DataLoader(Subset(dataset_xgb,valid_inds), batch_size=len(valid_inds), shuffle=False, num_workers=0)
    test_loader_xgb  = DataLoader(Subset(dataset_xgb, test_inds), batch_size=len(test_inds), shuffle=False, num_workers=1) 

  if USE_GRU: 
      train_loader = DataLoader(Subset(dataset,train_inds), batch_size=32, shuffle=True, num_workers=1)
      valid_loader = DataLoader(Subset(dataset,valid_inds), batch_size=len(valid_inds), shuffle=False, num_workers=1)
      test_loader  = DataLoader(Subset(dataset, test_inds), batch_size=len(test_inds), shuffle=False, num_workers=1)


if TEST:
  if USE_GRU:
    test_loader  = DataLoader(dataset, batch_size=len(dataset), shuffle=False, num_workers=1)
  if USE_XGB:
    test_loader_xgb  = DataLoader(dataset_xgb, batch_size=len(dataset_xgb), shuffle=False, num_workers=1)
print("Done!")

# **3**. Training the models

## **3.1** (Stacked) (Bi-) GRU model

Takes one source sequence as input.

Example of how to run the model.

In [ ]:
# TODO: define the constants upfront; explain what this cell does
N_EPOCHS=20 # maximum number of epochs to train the model
PATIENCE=3 # constant that controls the Early Stopping mechanism of the grid search for the GRU model

# make sure it is a list
if type(SUBREDDITS) != list:
  SUBREDDITS = list(SUBREDDITS)

if TRAIN:
  if len(SUBREDDITS) == 4:
    SRC_DATASET='all'
  if len(SUBREDDITS) == 1:
    SRC_DATASET=SUBREDDITS[0]

if TEST:
  SRC_DATASET='all'
#   SRC_DATASET='whatisthisthing'  
#   SRC_DATASET=SUBREDDITS[0] # use model trained with own dataset

PREFIX ='random'
if STRATIFIED:
  PREFIX += '_strat'

results_filename=f'{RESULTSDIR}{PREFIX}_{SRC_DATASET}_b{int(2./BIN_WIDTH):02}_f{FILTERED}_n{N_EPOCHS}_p{PATIENCE}_mGRUv1.pkl'

#results_filename=f'{DATADIR}{PREFIX}_{SRC_DATASET}_b{int(2./BIN_WIDTH):02}_f{FILTERED}_n{N_EPOCHS}_p{PATIENCE}_mGRUv1.pkl'
print(f'Results file is {results_filename}')

In [ ]:
%%time
GRID_SEARCH = False
USE_PAPER_SETUP = True
assert GRID_SEARCH != USE_PAPER_SETUP

if TRAIN:

    # test_params
    if USE_PAPER_SETUP:
        hidden_size_list     = [64]
        bidirectional_list   = [False]
        nlayers_dropout_list = [(2,0.0)]
    if GRID_SEARCH:
        hidden_size_list     = [4, 16, 64]
        bidirectional_list   = [False, True]
        nlayers_dropout_list = [(1,0.)] + [(2,prob) for prob in [0., .1, .2, .5]]
        
    train_criteria=[WeightedL1Loss( subreddit2weights['all'],BIN_WIDTH, MIN_VALUE)]
    test_criteria =[WeightedL1Loss( subreddit2weights['all'],BIN_WIDTH, MIN_VALUE),
              WeightedMSELoss(subreddit2weights['all'],BIN_WIDTH, MIN_VALUE),
              nn.L1Loss(), nn.MSELoss()]

    results_df = grid_search_train(train_loader,valid_loader,hidden_size_list, bidirectional_list, nlayers_dropout_list, train_criteria, test_criteria, results_filename,EMBEDDING_DIM=EMBEDDING_DIM, PLOT=False,N_EPOCHS=N_EPOCHS,PATIENCE=PATIENCE)

In [ ]:
# show best results
print(f'Loading {results_filename}...')
results_df = pd.read_pickle(results_filename)

results_df_view = pd.concat((results_df,results_df['params'].apply(pd.Series)), axis=1).drop(
    columns=['params','dropout_out','input_size','output_size','uses_two_series_as_input','para','para_best'])

results_df_view.sort_values('WeightedL1Loss').head(10)

In [ ]:
# plot relationship between losses
results_df.plot.scatter('WeightedL1Loss_best', 'WeightedL1Loss')

### Load the best model


In [ ]:
# load best model

RETRAIN = False
USE_LOSS_BEST = False
USE_BEST_VALIDATION = True
criterion = WeightedL1Loss(subreddit2weights['all'], BIN_WIDTH, MIN_VALUE)
#criterion = nn.MSELoss()
criterion_name = lambda x: x.__class__.__name__.split('.')[-1]


# see if the file with best params is available
# results_df = pd.read_pickle(results_filename)

if USE_LOSS_BEST:
    best_result = results_df.loc[results_df[criterion_name(criterion)+'_best'].argmin()]
else:
    best_result = results_df.loc[results_df[criterion_name(criterion)].argmin()]

# uncomment to load specific model instead
# best_result = results_df.iloc[0]
if RETRAIN:
    reset_seeds()
    model = models.GRUSentiment(best_result.params)
    _, valid_loss, _ = train_over_nepochs( model, train_loader, valid_loader,
                                        criterion=criterion, device=device,
                                        patience=3, n_epochs=N_EPOCHS)

    if USE_BEST_VALIDATION:
        model.load_state_dict(torch.load('checkpoint.pt', map_location=lambda storage, loc: storage))

else:
    model = models.GRUSentiment(best_result.params)
    if USE_BEST_VALIDATION:
        model.load_state_dict(best_result.para)
    else:
        model.load_state_dict(best_result.para_best)

model.to(device)

test_loss, outputs = evaluate(model, iter(test_loader), criterion=criterion, device=device, return_predictions=True)
model_yhat = outputs[0][0].cpu().numpy() # extract data from outputs
ilocs = outputs[0][2].cpu().numpy()
model_series = pd.Series(model_yhat.ravel(),index=ilocs)
# del outputs

# print model params
print(best_result.params)
print(f'Test loss: {test_loss:.3f}')

In [ ]:
tmp_df = get_baselines_df(test_loader, score_s, score_m)
tmp_df['model'] = model_series
tmp_df.head()

## **3.2** Training XGBoost Regressor as a baseline

In [ ]:
from src.xgb_utils import weightedl1_obj, weightedl1_loss_sklearn, weightedl1_loss_xgb, plot_model_error, hyperParameterTuning_xgb, getKeysByValue, grid_search_xgb,  

### XGB optimization

In [ ]:
GRIDS_MSE = True
GRIDS_L1 = False
if USE_XGB:
    if TRAIN:
        # LOADING DATA
        print("Loading data...")
        batch_train = next(iter(train_loader_xgb))
        batch_val   = next(iter(valid_loader_xgb))
        batch_test  = next(iter(test_loader_xgb))

        X_train,_,y_train,_ = batch_train
        X_val,  _,y_val,  _ = batch_val
        X_test, _,y_test, _ = batch_test

        # CONVERTING TO NUMPY ARRAYS
        X_train = X_train.to('cpu').numpy()
        y_train = y_train.to('cpu').numpy()
        X_val   = X_val.to('cpu').numpy()
        y_val   = y_val.to('cpu').numpy()
        X_test  = X_test.to('cpu').numpy()
        y_test  = y_test.to('cpu').numpy()   

        # DEFINING WEIGHTS
        bin_weights = subreddit2weights['all'].cpu().numpy()
        eval_set = [(X_train,y_train),(X_test,y_test),(X_val,y_val)]

        if GRID_SEARCH_XGB:
            param_tuning_xgb_l1 = {
                'learning_rate': [0.001,0.01, 0.1],
                'max_depth': [1, 3, 5],
                'min_child_weight': [1, 3, 5],
                'subsample': [0.5, 0.7],
                'colsample_bytree': [0.5, 0.7],
                'n_estimators' : [100, 200, 500],
                'objective': [weightedl1_loss_xgb]
            }
            param_tuning_xgb_mse = {
                'learning_rate': [0.001,0.01, 0.1],
                'max_depth': [1, 3, 5],
                'min_child_weight': [1, 3, 5],
                'subsample': [0.5, 0.7],
                'colsample_bytree': [0.5, 0.7],
                'n_estimators' : [100, 200, 500],
                'objective': ['reg:squarederror']
            }
            if GRIDS_L1:
                print("Starting grid search...")
                best_parameters_l1 = hyperParameterTuning_xgb(
                    XGBRegressor(objective=weightedl1_loss_xgb),
                    param_tuning_xgb_l1,X_train,y_train,X_val=X_val,y_val=y_val,
                    early_stop=3,eval_metric=weightedl1_loss_sklearn)
                print(f"Best parameters for the L1 model found by grid search:{best_parameters_l1}") 
                best_para_xgb = open(MODELSDIR + "best_para_xgb_l1.pkl", "wb")
                pickle.dump(best_parameters_l1, best_para_xgb)
                best_para_xgb.close()    

                print("Fitting the model with WeightedL1Loss")
                xgb_model_l1 = XGBRegressor(**best_parameters_l1)
                xgb_model_l1.n_jobs = 4
                xgb_model_l1.fit(X_train,y_train,eval_set=eval_set,early_stopping_rounds=3,eval_metric = weightedl1_loss_sklearn)
                
                print(f"Best ntree_limit for the L1 model:{xgb_model_l1.best_ntree_limit}")
                f = open(MODELSDIR + "best_ntree_l1.txt", "w")
                f.write(str(xgb_model_l1.best_ntree_limit))
                f.close()

                xgb_model_l1.save_model(MODELSDIR + "xgb_l1.model")    

                if PLOT_XGB:
                    #WL1 MODEL
                    plot_model_error(xgb_model_l1,'WeightedL1')   

                #EVAL
                print("Evaluating model...")
                test_criteria =[WeightedL1Loss( subreddit2weights['all'],BIN_WIDTH,MIN_VALUE),
                                nn.L1Loss(), nn.MSELoss()] 

                print("L1Loss Model test error:")
                for criterion in test_criteria:
                    test_error = criterion(torch.FloatTensor(xgb_model_l1.predict(X_test)).to(device),torch.FloatTensor(y_test).to(device))
                    print(criterion.__class__.__name__,test_error)   

            if GRIDS_MSE:

                print("Starting grid search...")
                best_parameters_mse = grid_search_xgb(early_stop=3,X_train=X_train,y_train=y_train,X_val=X_val,y_val=y_val,n_jobs=4,MODELSDIR=MODELSDIR)
                print(f"Best parameters for the MSE model found by grid search:{best_parameters_mse[0]}") 
                best_para_xgb = open(MODELSDIR + "best_para_xgb_mse.pkl", "wb")
                pickle.dump(best_parameters_mse, best_para_xgb)
                best_para_xgb.close()

                print("Fitting the model with MSE")
                xgb_model_mse = XGBRegressor(**best_parameters_mse)
                xgb_model_mse.n_jobs = 4

                xgb_model_mse.fit(X_train,y_train,eval_set=eval_set,early_stopping_rounds=3)
                print(f"Best ntree_limit for the MSE model:{xgb_model_mse.best_ntree_limit}")
                f = open(MODELSDIR + "best_ntree_mse.txt", "w")
                f.write(str(xgb_model_mse.best_ntree_limit))
                f.close()

                xgb_model_mse.save_model(MODELSDIR + "xgb_mse.model")    

                if PLOT_XGB:
                    #MSE MODEL
                    plot_model_error(xgb_model_mse,'rmse')      

                #EVAL
                print("Evaluating model...")
                test_criteria =[WeightedL1Loss( subreddit2weights['all'],BIN_WIDTH,MIN_VALUE),
                                nn.L1Loss(), nn.MSELoss()] 

                print("MSE Model test error:")
                for criterion in test_criteria:
                    test_error = criterion(torch.FloatTensor(xgb_model_mse.predict(X_test)).to(device),torch.FloatTensor(y_test).to(device))
                    print(criterion.__class__.__name__,test_error)
            
        else:
            print("Loading best parameters...")
            try:
                best_parameters_l1 = open(MODELSDIR + "best_para_xgb_l1.pkl", "rb")
                best_parameters_l1 = pickle.load(best_parameters_l1)
                print("Successfully loaded best parameters for the L1 model")
            except:
                print("L1 model parameters file not found, using standard ones instead")
                best_parameters_l1 = {
            'learning_rate': 0.1,
            'max_depth':  5,
            'min_child_weight': 3,
            'subsample': 0.5,
            'colsample_bytree': 0.5,
            'n_estimators' :  200,
            'objective': weightedl1_loss_xgb}

            try:
                best_parameters_mse = open(MODELSDIR + "best_para_xgb_mse.pkl", "rb")
                best_parameters_mse = pickle.load(best_parameters_mse)
                print("Successfully loaded best parameters for the MSE model")
            except:
                print("MSE model parameters file not found, using standard ones instead")
                best_parameters_mse = {
            'learning_rate': 0.1,
            'max_depth':  5,
            'min_child_weight': 3,
            'subsample': 0.5,
            'colsample_bytree': 0.5,
            'n_estimators' :  200,
            'objective': 'reg:squarederror'}            


            print("Fitting the model with WeightedL1Loss")
            xgb_model_l1 = XGBRegressor(**best_parameters_l1, n_jobs = 4)
            xgb_model_l1.fit(X_train,y_train,eval_set=eval_set,early_stopping_rounds=3)#,eval_metric = WeightedL1Loss_calc)

            print(f"Best ntree_limit for the model:{xgb_model_l1.best_ntree_limit}")
            f = open(MODELSDIR + "best_ntree_l1.txt", "w")
            f.write(str(xgb_model_l1.best_ntree_limit))
            f.close()

            xgb_model_l1.save_model(MODELSDIR + "xgb_l1.model")

            print("Fitting the model with MSE")
            xgb_model_mse = XGBRegressor(**best_parameters_mse, n_jobs=4)
            xgb_model_mse.fit(X_train,y_train,eval_set=eval_set,early_stopping_rounds=3)

            print(f"Best ntree_limit for the model:{xgb_model_mse.best_ntree_limit}")
            f = open(MODELSDIR + "best_ntree_mse.txt", "w")
            f.write(str(xgb_model_mse.best_ntree_limit))
            f.close()

            xgb_model_mse.save_model(MODELSDIR + "xgb_mse.model")     

            if PLOT_XGB:
                #WL1 MODEL
                plot_model_error(xgb_model_l1,'WeightedL1Loss') 

                #MSE MODEL
                plot_model_error(xgb_model_mse,'rmse')

            #EVAL
            print("Evaluating model...")
            print("-------------------------")
            test_criteria =[WeightedL1Loss( subreddit2weights['all'],BIN_WIDTH,MIN_VALUE),
                            nn.L1Loss(), nn.MSELoss()] 
                            
            print("MSE Model test error:")
            for criterion in test_criteria:
                test_error = criterion(torch.FloatTensor(xgb_model_mse.predict(X_test)).to(device),torch.FloatTensor(y_test).to(device))
                print(criterion.__class__.__name__,test_error)

            print("-------------------------")
            print("L1Loss Model test error:")
            for criterion in test_criteria:
                test_error = criterion(torch.FloatTensor(xgb_model_l1.predict(X_test)).to(device),torch.FloatTensor(y_test).to(device))
                print(criterion.__class__.__name__,test_error)


    if TEST:
        # LOADING DATA
        batch_test = next(iter(test_loader_xgb))
        X_test,_,y_test,_ = batch_test
        # CONVERTING TO NUMPY ARRAYS
        X_test = X_test.to('cpu').numpy()
        y_test = y_test.to('cpu').numpy() 
        
        #LOADING MODELS
        print("Loading trained xgb model...")
        xgb_model_l1 = XGBRegressor()
        xgb_model_mse = XGBRegressor()
        xgb_model_l1.load_model(MODELSDIR + "xgb_l1.model")
        xgb_model_mse.load_model(MODELSDIR + "xgb_mse.model")
        
        
        best_ntree_l1 = open(MODELSDIR + "best_ntree_l1.txt", "r")
        best_ntree_l1 = int(best_ntree_l1.read())
        best_ntree_mse = open(MODELSDIR + "best_ntree_mse.txt", "r")
        best_ntree_mse = int(best_ntree_mse.read())  

        print("Model and parameters loaded")
        print("-------------------------")

        #EVAL
        print("Evaluating model...")
        print("-------------------------")
        test_criteria =[WeightedL1Loss( subreddit2weights['all'],BIN_WIDTH,MIN_VALUE),
                        nn.L1Loss(), nn.MSELoss()] 
                        
        print("MSE Model test error:")
        for criterion in test_criteria:
            test_error = criterion(torch.FloatTensor(xgb_model_mse.predict(X_test)).to(device),torch.FloatTensor(y_test).to(device))
            print(criterion.__class__.__name__,test_error)

        print("-------------------------")
        print("L1Loss Model test error:")
        for criterion in test_criteria:
            test_error = criterion(torch.FloatTensor(xgb_model_l1.predict(X_test)).to(device),torch.FloatTensor(y_test).to(device))
            print(criterion.__class__.__name__,test_error)


## **4**. Comparing the results with simple baselines

In [ ]:

def compute_error(results_df, pred_column_name, criteria, device):
  return {criterion.__class__.__name__.split('.')[-1]:
          float(criterion(
              torch.Tensor(results_df[pred_column_name].values).to(device),
              torch.Tensor(results_df['final score'].values).to(device))
          ) for criterion in criteria}

In [ ]:
# TODO: explain what this cell does
pred_column_names = ['unchanged', 'mean','last']
loss2title = {'L1Loss':'L1 Loss','MSELoss':'MSE Loss','WeightedL1Loss':'Weighted L1 Loss'}
subreddit='all'

criteria=[WeightedL1Loss(subreddit2weights[subreddit],BIN_WIDTH,MIN_VALUE)]
criteria_names = [criterion.__class__.__name__.split('.')[-1] for criterion in criteria]

test_loss = dict()
for pred_column_name in pred_column_names:
    test_loss[pred_column_name] = compute_error(tmp_df, pred_column_name, criteria, device)

df = pd.DataFrame.from_dict(test_loss,orient='index')
df.columns = df.columns.map(loss2title)
df.index = df.index.map(str.upper)
df

### Generate results table

In [ ]:
if TRAIN:
    subreddit2range2 = {}
    for subreddit,v in subreddit2range.items():
        if type(v) == slice:
            inds = [ind for ind in tmp_df.index if v.start <= ind < v.stop]
        else:
            inds = list(tmp_df.index)
        subreddit2range2[subreddit] = inds
elif TEST:
    test_locs_df = pd.DataFrame(test_locs)
    subreddit2range2 = {'all': test_locs_df.index}
    for subreddit in SUBREDDITS:
      subreddit2range2[subreddit] = test_locs_df[test_locs_df[0] == subreddit].index


pred_column_names = ['unchanged', 'mean','last','model']
loss2title = {'L1Loss':'L1 Loss','MSELoss':'MSE Loss','WeightedL1Loss':'Weighted L1 Loss'}

In [ ]:
tmp_df.head()

In [ ]:
# tmp_df['model'] = model_yhat
# tmp_df['xgboost'] = xgb_model_l1.predict(X_test)
# tmp_df['xgb-mse'] = xgb_model_mse.predict(X_test)


df_list = []
for subreddit in ['all']+SUBREDDITS:
  criteria=[WeightedL1Loss(subreddit2weights[subreddit],BIN_WIDTH,MIN_VALUE)]
  #criteria=[nn.L1Loss(),nn.MSELoss()]
  criteria_names = [criterion.__class__.__name__.split('.')[-1] for criterion in criteria]

  test_loss = dict()
  for pred_column_name in pred_column_names:
    test_loss[pred_column_name] = compute_error(tmp_df.loc[subreddit2range2[subreddit]], pred_column_name, criteria, device)

  df = pd.DataFrame.from_dict(test_loss,orient='index')
  df.columns = df.columns.map(loss2title)
  df.index = df.index.map(str.upper)
  df_list.append(df)

test_df = pd.concat(df_list,keys=['ALL']+[subreddit2title[subreddit.lower()] for subreddit in SUBREDDITS],axis=1)
test_df = test_df.swaplevel(0,1,axis=1).sort_index(1)
n=len(SUBREDDITS)+1
test_df = test_df.round(3)
test_df = test_df.loc[['UNCHANGED','MEAN','LAST','MODEL'],:]
test_df

In [ ]:
formatters = []
for column in test_df.columns:
  formatters.append(lambda x, column=column: '\\textbf{%s}'%( ('%.3f' % float(x)).lstrip('0') ) if float(x) == test_df[column].min() else ('%.3f'% float(x)).lstrip('0') )
formatters

In [ ]:
print(test_df.to_latex(
    escape=False,column_format='l|ccccc|ccccc|ccccc',multicolumn_format='c',
    label='tab:test_loss', formatters=formatters,
    caption='Results w.r.t.\\ L1, MSE and proposed Weighted L1 Loss, which gives more weight to more extreme responses. '+
    'Model is trained on entire 2017 dataset whereas \\textsc{Model-Subreddit} is trained on target subreddit (column). '+
    '\\textsc{Mean} yields lowest L1 and MSE, but Model outperforms baselines w.r.t.\\ Weighted L1 loss.'
    ))

In [ ]:
pred_column_names2 = ['final score']+pred_column_names
fig, axs = plt.subplots(1,len(pred_column_names2), figsize=(3*len(pred_column_names2),3))
for ax, column_name in zip(axs,pred_column_names2):
  ax = tmp_df[column_name].hist(ax=ax,bins=np.arange(-1.0,1.0,.1))
  ax.set_xlim(-1,1)
  ax.set_ylim(0,700)
  ax.set_title(column_name)

# **5**. Plotting the results

In [ ]:
# TODO: add doc to function
def plot_prediction_density(tmp_df, pred_column_names, subreddit=None, axs=None, choice_inds=None, cmap=plt.cm.gist_earth_r, vmax=None):
  if subreddit is not None:
    tmp_df = tmp_df.loc[subreddit2range2[subreddit]]

  xmin = ymin = -1.
  xmax = ymax =  1.

  # deltaX = (max(x) - min(x))/10
  # deltaY = (max(y) - min(y))/10
  # xmin = min(x) - deltaX
  # xmax = max(x) + deltaX
  # ymin = min(y) - deltaY
  # ymax = max(y) + deltaY
  # xx, yy = np.mgrid[xmin:xmax:50j, ymin:ymax:50j]

  xx, yy = np.mgrid[xmin:xmax:50j, ymin:ymax:50j]
  positions = np.vstack([xx.ravel(), yy.ravel()])
  reset_seeds()

  if choice_inds is None:
    choice_inds = np.random.choice(len(tmp_df), 100)
  print(tmp_df.iloc[choice_inds]['thread ncom'].max())
  # choice_inds = np.random.choice(nz[0], 100)

  if axs is None:
   fig, axs = plt.subplots(2,2,figsize=(3*2,3*2),sharex=True,sharey=True)
   plt.subplots_adjust(wspace=.15,hspace=.15)

  for idx, (ax, column_name) in enumerate(zip(axs.ravel(),pred_column_names)):
    loss_array = tmp_df[['final score', column_name]].values
    kernel = stats.gaussian_kde(loss_array.T)
    Z = np.reshape(kernel(positions).T, xx.shape)
    # print(subreddit,idx,Z.min(),Z.max())

    if vmax is not None:
      levels = np.linspace(vmax/8.,vmax,8)
      # levels = np.insert(np.linspace(vmax*2/7.,vmax,7),0,0)
      vmin=vmax/8.
    else:
      levels = 8
      vmin=None

    ax.plot([xmin, xmax], [ymin, ymax], ls='-', c='k',alpha=.5)#,transform=ax.transAxes)

    # cfset = ax.contourf(xx, yy, Z, cmap=cmap, levels=levels)
    hb = ax.imshow(np.rot90(Z), cmap=cmap, aspect='auto',
              extent=[xmin, xmax, ymin, ymax], vmin=0,vmax=vmax)
    # cset = ax.contour(xx, yy, Z, colors='k', linewidths=.25, levels=levels)
    # plt.colorbar(hb, ax=ax)
    # ax.clabel(cset, inline=1, fontsize=8)


    ncom = tmp_df['branch ncom'].values if column_name.startswith('branch') else tmp_df['thread ncom'].values
    ax.scatter(loss_array[choice_inds,0], loss_array[choice_inds,1], 2*ncom[choice_inds], 'gray', edgecolor='k')

    ax.set_xlim([xmin, xmax])
    ax.set_ylim([ymin, ymax])
    # ax.yaxis.tick_right()
    # ax.set_xticks(np.arange(-1.,1.,.5))
    ax.set_yticks(np.arange(-1.,1.01,.5))
    ax.grid(ls='--', c='gray', alpha=.5)

    # ax.axhline((ymin+ymax)/2,color='k',alpha=.5)
    # ax.axvline((xmin+xmax)/2,color='k',alpha=.5)
    # x = np.array([xmin,xmax])
    # ax.plot(x, 1.5*x, ls='--', c='k',alpha=.5)
    # ax.plot(x, 0.5*x, ls='--', c='k',alpha=.5)
    # ax.set_title(column_name)


  # divider = make_axes_locatable(plt.gca())
  # cax = divider.append_axes("right", size="2%", pad=0.00)
  # cbar = fig.colorbar(hb, cax=cax)

  # fig.text(0.5, 0.04, r'EmT($c_n$) (true value)', ha='center')
  # fig.text(0.04, 0.5, 'prediction', va='center', rotation='vertical')

  # plt.savefig(RESULTSDIR+subreddit+'.pdf')
  return hb

In [ ]:
# plot density
vmax_list = [2, 3.2, 2., 2.]
vmax_list = [2.4, 2.4, 2.4, 2.4]

choice_inds = None

fig, axs = plt.subplots(4,4,figsize=(3*4,3*4),sharex=True,sharey=True)
plt.subplots_adjust(wspace=.1,hspace=.1)

for ix, subreddit in enumerate(SUBREDDITS):
  hb = plot_prediction_density(tmp_df, pred_column_names[1:], subreddit, axs[ix,:], choice_inds=choice_inds, vmax=vmax_list[ix])

fig.text(0.5 , 0.08, r'EmT($c_n$) (true value)', ha='center', fontsize='x-large')
fig.text(0.0, 0.5 , 'prediction', va='center', rotation='vertical', fontsize='x-large')

cbaxes = fig.add_axes([0.2, 0.93, .6, 0.01])

for ax, col in zip(axs[0], pred_column_names[1:]):
  ax.set_title(col.upper(), size='large')

for ax, row in zip(axs[:,0], SUBREDDITS):
  # ax.yaxis.set_label_position("right")
  ax.set_ylabel(subreddit2title[row.lower()], rotation=0, size='large', horizontalalignment='right')

cb = plt.colorbar(hb, cax = cbaxes, orientation='horizontal')
cb.ax.set_title('density', size='large')
plt.savefig('heatmaps.pdf')

# fig.colorbar(hb, ax=axs,location='bottom', shrink=.5)
# plt.tight_layout()

# divider = make_axes_locatable(plt.gca())
# cax = divider.append_axes("right", size="2%", pad=0.00)
# cbar = fig.colorbar(hb, cax=cax)


# **6**. Case Study

In [ ]:
# TODO: explain what we are reading
REDDIT='SuicideWatch'
suffix = '.pkl'
with open(DATADIR+REDDIT+'_post2data'+suffix,'rb') as infile:
  post2data = pickle.load(infile)

with open(DATADIR+REDDIT+'_comment2data'+suffix,'rb') as infile:
  comment2data = pickle.load(infile)

In [ ]:
# 5855 (original iloc = 58762)
z = tmp_df.loc[subreddit2range2['SuicideWatch']]
z[(z['final score'] > 0.75) & (z['model'] < -0.75)].sort_values('final score',ascending=True).head(20)

In [ ]:
# 374 (original iloc = 4001)
z = tmp_df.loc[subreddit2range2['Anxiety']]
z[(z['final score'] > 0.75) & (z['model'] > 0.6)].sort_values('thread ncom',ascending=True).head(20)

In [ ]:
def print_post_and_comments(index, suffix='', infile_extension='.pkl'):
  post = post_df.iloc[index]

  print(f'Post id: {post.name[1]}')
  print(f'Author (score {(post["score"][0]-0.5)*2}):')

  subreddit = post.name[0]
  with open(f'{DATADIR}{subreddit}_comment2data{suffix}{infile_extension}','rb') as infile:
    comment2data = pickle.load(infile)

  comment_ids = post.comments
  pprint.pprint(post.text)
  for cx in range(max(0,post.seq_len-10),post.seq_len-1):
    print('Author' if post.is_post_author[cx+1] else 'Commenter', end='')
    print(f' (score {(post["score"][cx+1])}):')
    if 'body' in comment2data[comment_ids[cx]]:
      pprint.pprint(comment2data[comment_ids[cx]]['body'])
    else:
      pprint.pprint(comment2data[comment_ids[cx]]['text'])

In [ ]:
# TODO: remove for in all cells below, just set ind to each value
#for ind in [5855]: [58762]:
ind = np.where(post_df.reset_index().level_1 == '712ctw')[0][0]
print(post_df.iloc[ind])
print(tmp_df.loc[ind])
print_post_and_comments(ind)

In [ ]:
# 1336 (original iloc = 13443)
z = tmp_df.loc[subreddit2range2['bipolar']]
z[(z['final score'] < -0.75) & (z['model'] < 0.1) & (z['model'] > 0)].sort_values('thread ncom',ascending=False).head(20)

In [ ]:
# 1422 (original iloc = 14185)
z[(z['final score'] > -0.5) & (z['model'] < -0.5)].sort_values('thread ncom',ascending=False).head(20)

In [ ]:
#  5090 (original iloc = 50549)
z = tmp_df.loc[subreddit2range2['SuicideWatch']]
z[(z['final score'] > .6) & (z['final score'] < .75)  & (z['model'] > .6) & (z['model'] < .75)].sort_values('thread ncom',ascending=False)


In [ ]:
# for ind in [5090]: [50549]:
ind = np.where(post_df.reset_index().level_1 == '5nr93y')[0][0]
print(post_df.iloc[ind])
print(tmp_df.loc[ind])
print_post_and_comments(ind)

In [ ]:
#for ind in [13443]:
ind = np.where(post_df.reset_index().level_1 == '5x1ll5')[0][0]
print(post_df.iloc[ind])
print(tmp_df.loc[ind])
print_post_and_comments(ind)
print('------------------------------------------------------')

# **7**. Can we predict large fluctuations from one author comment to the next?

1. Plot the distribution of changes from one comment to the next.
2. Take the largest fluctuations and see how well we predict those. Is this large or small? Are the baselines predicting those better?
3. For those that we do well, inspect content of the post, "last checkpoint of author", following comment and last comment of author.

In [ ]:
#TODO: alternate the cells with the descriptions 1,2,3 above
fluctuations = []


for batch in test_loader:
  is_author_value = batch[0][0][0,-1] # this has to be retrieved because the feature was z-normalized
  batch_size = batch[0].size(0)

for ix in range(batch_size):
    ncom = batch[1][ix]
    for jx in range(ncom-1,-1,-1):
      if batch[0][ix][jx,-1] == is_author_value:
          fluctuations.append(batch[2][ix]-(batch[0][ix][jx,-2]*score_s+score_m))
          break   

fluctuations = np.array(fluctuations)
# fluctuation_series = pd.Series(fluctuations, index=test_locs)
fluctuation_series = pd.Series(fluctuations)
fluctuation_series.hist()

In [ ]:
fluctuation_series = tmp_df['final score']-tmp_df['unchanged']
fluctuation_series.head()

In [ ]:
# positive variation



def get_error_stats(series, lower_bound=None,upper_bound=None):
  assert (lower_bound is not None) ^ (upper_bound is not None)
  if upper_bound is not None:
    selected_inds = series[series < upper_bound].index
    print(f'[Upper bound: {upper_bound:.3f}] ',sep='')
  if lower_bound is not None:
    selected_inds = series[series > lower_bound].index
    print(f'[Lower bound: {lower_bound:.3f}] ',sep='')

  sub_df = tmp_df.loc[selected_inds]

  model_error = (sub_df['final score']-sub_df['model']).abs()
  results = {'MODEL L1':model_error.mean()}
  for baseline in ['mean','last','xgb']:
    baseline_error = (sub_df['final score']-sub_df[baseline]).abs()
    succ_fraction = (model_error < baseline_error).mean()
    print( f"Model outperforms {baseline} in {succ_fraction*100:.1f}% of all cases")
    print( f"Model error: {model_error.mean():.3f}\t Baseline error: {baseline_error.mean():.3f}")
    results[baseline] = {f'M outp. {baseline.upper()} (%)': succ_fraction,
                         f'{baseline.upper()} L1': baseline_error.mean() }

  print(f"Cases: {len(selected_inds)}\n\n")
  return results

n=len(fluctuation_series)
upper_bound=fluctuation_series.sort_values().iloc[int(0.05*n)]
lower_bound=fluctuation_series.sort_values().iloc[int(0.95*n)]

extreme_stats = {}

print('XXXX Fluctuation from previous comment: <5-th and >95-th percentile')
extreme_stats['EmT shift $>$ 95th perc.'] = get_error_stats(fluctuation_series, lower_bound=lower_bound)
extreme_stats['EmT shift $>$ +1.0'] = get_error_stats(fluctuation_series, lower_bound=1.)
extreme_stats['Final EmT $>$ +0.8'] = get_error_stats(tmp_df['final score'], lower_bound=.8)

extreme_stats['EmT shift $<$  5th perc.'] = get_error_stats(fluctuation_series, upper_bound=upper_bound)
extreme_stats['EmT shift $<$ -1.0'] = get_error_stats(fluctuation_series, upper_bound=-1.)
extreme_stats['Final EmT $<$ -0.8'] = get_error_stats(tmp_df['final score'], upper_bound=-.8)

print('XXX Fluctuation from previous comment: <-0.5 and >0.5')

print('XXX Final score: <0.1 and >0.9')


In [ ]:
extreme_df = pd.DataFrame.from_dict(extreme_stats, orient='index')
extreme_df = pd.concat((extreme_df,extreme_df['mean'].apply(pd.Series),extreme_df['last'].apply(pd.Series),extreme_df['xgb'].apply(pd.Series)), axis=1).drop(columns=['last','mean','xgb'])
#extreme_df[['MEAN L1','MODEL L1','LAST L1','XGB L1']] = extreme_df[['MEAN L1','MODEL L1','LAST L1']].multiply(2)
extreme_df = extreme_df.round(3)
extreme_df['M outp. MEAN (%)'] = extreme_df['M outp. MEAN (%)']*100
extreme_df['M outp. LAST (%)'] = extreme_df['M outp. LAST (%)']*100
extreme_df['M outp. XGB (%)'] = extreme_df['M outp. XGB (%)']*100

new_columns = [extreme_df.columns[ix] for ix in [0,2,4,6,1,3,5]] 
extreme_df = extreme_df[new_columns]
extreme_df

In [ ]:
formatters = []
for ix, column in enumerate(extreme_df.columns):
  formatters.append(lambda x, column_ix=ix: '\\textbf{%s}'%( ('%.1f' % float(x)).lstrip('0') ) if (column_ix >=3 and x > 50) else ('%.3f'% float(x)).lstrip('0') )
formatters

In [ ]:
print(extreme_df.to_latex(
    column_format='l|rrrr|ccc',  escape=False, label='tab:extreme_values', formatters=formatters,
    caption='Performance on extreme values. Central columns show L1 error for \\textsc{Mean}, \\textsc{Model} and \\textsc{Last}. '+
    'First (last) column show fraction of cases (selected from test set according to row description) where \\textsc{Model} outperforms '+
    '\\textsc{Mean} (\\textsc{Last}). For extremely positive shifts, \\textsc{Last} performs best because a large improvement is often '+
    'preceded by a very positive comment. For extremely negative shifts, \\textsc{Model} performs best.'))

In [ ]:
for ind in large_change_inds:
  print(ind, branch_data[ind])
  index, _, bx = branch_data[ind]
  b = post_df.loc[index].valid_branches[bx]
  print(index, bx, b)
  print(post_df.loc[index].features[b,-2])
  print(post_df.loc[index].features[b,-1])


In [ ]:
for pred_column_name in pred_column_names:
  test_loss[pred_column_name] = compute_error(tmp_df.iloc[large_change_inds], pred_column_name, criteria)
pd.DataFrame.from_dict(test_loss, orient='index').round(3)

In [ ]:
attn_weights = attn_weights.squeeze(1).cpu().numpy()

In [ ]:
count_nz = np.count_nonzero( attn_weights,  axis=1)

In [ ]:
attn_weights_sorted = -np.sort(-attn_weights, axis=1)
imbalance = (attn_weights_sorted[:,0]-attn_weights_sorted[:,1])
imbalance_index = np.argsort(-imbalance)
imbalance_index[0]

In [ ]:
X = np.vstack((pred_error,imbalance,initial_scores,y,yhat,te_diff, count_nz))
X = X.T
tmp_df = pd.DataFrame(X, columns=['pred_error','imbalance','initial_score','final_score','prediction','te_diff', 'count_nz'], index=post_ids)
tmp_df.tail()

In [ ]:
pred_error = np.absolute(z[:,0]-z[:,1])
cost = pred_error-2*imbalance.cpu().numpy()-0.6*np.absolute(z[:,1])-0.1*(attn_weights[:,0,2].cpu().numpy()>0)
index_array = np.argsort(cost)
index_array[:10]

In [ ]:
index = index_array[2]
print(z[index])
attn_weights[index]

In [ ]:
post_id = post_ids[index]
post=post_df.loc[post_id]
print(post)
#pprint.pprint(post['text'])

In [ ]:
comment = comment_df.loc[post_df.loc[post_id,'comments'][7]]
print(comment) 
pprint.pprint(comment['text'])